<a href="https://colab.research.google.com/github/TaniaZharova2205/HSE/blob/main/Chocolate_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Baseline-решение

## Импорт библиотек, загрузка данных

In [1]:
import pandas as pd
import numpy as np

In [2]:
TRAIN = "https://raw.githubusercontent.com/evgpat/edu_stepik_practical_ml/main/datasets/chocolate_train.csv"
TEST = "https://raw.githubusercontent.com/evgpat/edu_stepik_practical_ml/main/datasets/chocolate_test_new.csv"

In [3]:
train_df = pd.read_csv(TRAIN)

## Обзор данных, подготовка к обучению

In [4]:
train_df.head()

,Company,Specific Bean Origin,REF,Review,Cocoa Percent,Company Location,Rating,Bean Type,Broad Bean Origin
0,Willie's Cacao,Rio Caribe,457,2009,72%,U.K.,3.25,Trinitario,Venezuela
1,Beschle (Felchlin),"Ocumare, Premier Cru, Quizas No. 2",508,2010,72%,Switzerland,3.50,,Venezuela
2,Dark Forest,Tanzania,1554,2015,70%,U.S.A.,3.00,,Tanzania
3,Brasstown aka It's Chocolate,Cooproagro,1125,2013,72%,U.S.A.,3.00,Trinitario,Dominican Republic
4,Pralus,"Java, Indonesie",32,2006,75%,France,3.50,Criollo,Indonesia


Удаляем все строки с пропусками

In [5]:
train_df.dropna(axis=0, inplace=True)

Выделяем матрицу объект-признак и вектор с целевой переменной

In [6]:
X_train = train_df.drop('Rating', axis=1)
y_train = train_df['Rating']

In [7]:
from sklearn.model_selection import train_test_split
Xtrain, Xval, ytrain, yval = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

## Обучение модели

Устанавливаем катбуст и обучаем его с гиперпараметрами по умолчанию на всех данных

In [8]:
!pip install catboost -q

In [9]:
Xtrain.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1002 entries, 199 to 1128
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Company               1002 non-null   object
 1   Specific Bean Origin  1002 non-null   object
 2   REF                   1002 non-null   int64 
 3   Review                1002 non-null   int64 
 4   Cocoa Percent         1002 non-null   object
 5   Company Location      1002 non-null   object
 6   Bean Type             1002 non-null   object
 7   Broad Bean Origin     1002 non-null   object
dtypes: int64(2), object(6)
memory usage: 70.5+ KB


In [10]:
cat_features = [0, 1, 4, 5, 6, 7]

In [11]:
from catboost import CatBoostRegressor
from sklearn.preprocessing import StandardScaler

num_features = ['REF', 'Review']
scaler = StandardScaler()
Xtrain[num_features] = scaler.fit_transform(Xtrain[num_features])
Xval[num_features] = scaler.transform(Xval[num_features])

In [13]:
from sklearn.model_selection import GridSearchCV

params = {
    'depth': [6, 8, 10],
    'learning_rate': [0.01, 0.03, 0.1],
    'l2_leaf_reg': [1, 3, 5],
    'iterations': [1000, 2000]
}

grid = GridSearchCV(
    CatBoostRegressor(cat_features=cat_features, loss_function='RMSE', verbose=0),
    param_grid=params,
    cv=5,
    scoring='r2',
    n_jobs=-1
)

grid.fit(Xtrain, ytrain)
print("Лучшие параметры:", grid.best_params_)

Лучшие параметры: {'depth': 8, 'iterations': 1000, 'l2_leaf_reg': 1, 'learning_rate': 0.01}


## Предсказание на тестовых данных

In [14]:
from sklearn.metrics import r2_score
pred = grid.predict(Xval)
r2_score(yval, pred)

0.11188073409276023

Загружаем тестовые данные

In [15]:
test_df = pd.read_csv(TEST)

In [16]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 540 entries, 0 to 539
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Company               540 non-null    object
 1   Specific Bean Origin  540 non-null    object
 2   REF                   540 non-null    int64 
 3   Review                540 non-null    int64 
 4   Cocoa Percent         540 non-null    object
 5   Company Location      540 non-null    object
 6   Bean Type             540 non-null    object
 7   Broad Bean Origin     540 non-null    object
dtypes: int64(2), object(6)
memory usage: 33.9+ KB


In [ ]:
scaler = StandardScaler()
X_train[num_features] = scaler.fit_transform(X_train[num_features])
test_df[num_features] = scaler.transform(test_df[num_features])
grid.fit(X_train, y_train)
print("Лучшие параметры:", grid.best_params_)

Делаем предсказание

In [ ]:
pred = grid.predict(test_df)

Сохраняем предсказание в файл в нужном формате

In [ ]:
test_df['id'] = np.arange(len(test_df))
test_df['Rating'] = pred

test_df[['id','Rating']].to_csv("baseline_submission.csv", index=False)